In [18]:
# import the opencv library
import cv2

In [19]:
# import our model
from keras.models import load_model

In [20]:
# Other dependencies
import numpy as np
import tensorflow as tf
import os
from PIL import Image

In [21]:
# Crop center of the image feed
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

# Crop largest square from the image feed
def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))

In [22]:
# define a video capture object
stream = cv2.VideoCapture(0)
model = load_model('emotion_model.h5')

while(True):
      
    # Capture the video frame by frame
    res, frame = stream.read()
    height, width, channel = frame.shape
    
    #Convert the captured frame into RGB
    im = Image.fromarray(frame, 'RGB')
    
    #Resizing into dimensions you used while training
    im_new = crop_max_square(im)
    im_new = im_new.resize((48,48))
    img_array = np.array(im_new)
    
    #Expand dimensions to match the 4D Tensor shape.
    img_array = np.expand_dims(img_array, axis=1)
    img_array = img_array.reshape(-1, 48, 48, 1)
    
    # Scale to match train data
    img_array = img_array/255
    
    #Calling the predict function using keras
    prediction = model.predict(img_array)
    print(prediction)
    
    # Taking only the first of three arrays
    array = prediction[0]
    
    # Find location of the highest probability
    predict = np.where(array == max(array))
    emotion = predict[0]
    
    # Part of the code that predicts the emotion from the prediction array
    if(emotion == 0):
        output = "Angry"
    elif(emotion == 1):
        output = "Disgust"
    elif(emotion == 2):
        output = "Fear"
    elif(emotion == 3):
        output = "Happy"
    elif(emotion == 4):
        output = "Sad"
    elif(emotion == 5):
        output = "Surprise"
    elif(emotion == 6):
        output = "Neutral"
    
    cv2.putText(frame, f'{output} {max(array)}%', (0,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    # Display the resulting frame
    cv2.imshow('Predict Emotion', frame)
      
    # the 'q' button is set as the quitting button
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
stream.release()

# Destroy all the windows
cv2.destroyAllWindows()

1/1 [==============================] - 0s 107ms/step
[[5.82598746e-02 2.52408772e-05 8.71027529e-04 5.86345553e-01
  1.24812670e-01 8.48024967e-04 2.28837654e-01]
 [1.63423631e-03 2.33302679e-04 9.53512063e-05 7.32935905e-01
  1.69675246e-01 7.58314356e-02 1.95946433e-02]
 [3.53222825e-02 1.47471364e-04 1.46289554e-03 9.18751359e-01
  3.16065289e-02 1.49486877e-04 1.25600565e-02]]
1/1 [==============================] - 0s 33ms/step
[[5.61111383e-02 2.81589600e-05 8.11104197e-04 5.78152061e-01
  1.43864304e-01 8.65907408e-04 2.20167324e-01]
 [1.43782375e-03 2.68923701e-04 1.06405205e-04 7.48025477e-01
  1.60887688e-01 7.05031157e-02 1.87705513e-02]
 [4.23106588e-02 1.96244102e-04 1.70577003e-03 9.11300540e-01
  3.13713476e-02 1.42304780e-04 1.29731949e-02]]
1/1 [==============================] - 0s 28ms/step
[[6.1635833e-02 2.5552636e-05 8.7131822e-04 5.4596627e-01 1.7566787e-01
  8.7144744e-04 2.1496163e-01]
 [1.3146386e-03 2.4378697e-04 9.7246244e-05 7.5928700e-01 1.5531656e-01
  6.55